In [1]:
import findspark

####TODO:  replace the original one
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
# findspark.init(r"C:\ProgramData\spark\spark-2.1.1-bin-hadoop2.7")
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('iteration4_a').getOrCreate()

In [2]:
import csv
import pandas as pd
import re
import io
import pandas as pd
import numpy as np

infile_2017 = "2017.csv"
infile_2016 = "2016.csv"
outfile_2017 = "strip_2017.csv"
outfile_2016 = "strip_2016.csv"


def clean_carriage_return(infile, outfile):
    write_header = True
    with open(infile, 'r' ,encoding='iso-8859-1') as src:
        data = src.read()
        df = pd.read_csv(io.StringIO(re.sub('"\s*\n','"',data)), chunksize=1000)   
        for chunk in df:
            chunk = chunk.replace('(?!(([^"]*"){2})*[^"]*$),', '', regex=True)
            clean_chunk = pd.DataFrame(chunk.replace({r'\r\n': ''}, regex=True))
            for col in clean_chunk.columns:
                if clean_chunk[col].dtype == np.object_:
                    clean_chunk[col] = clean_chunk[col].str.replace('\n','')
            clean_chunk.to_csv(outfile, sep = ",", index=False, mode='a', encoding='iso-8859-1',quotechar='"',
                        quoting=csv.QUOTE_NONNUMERIC,header=write_header )
            write_header = False # Header is marked as False, so that for all chunks we should have just one header in csv

clean_carriage_return(infile_2016, outfile_2016)
clean_carriage_return(infile_2017, outfile_2017)

In [3]:
df_16 = spark.read.format("csv").options(header="true", inferschema = "true").option("quote", "\"").option("escape", "\"").load("strip_2016.csv")
df_17 = spark.read.format("csv").options(header="true", inferschema = "true").option("quote", "\"").option("escape", "\"").load("strip_2017.csv")

In [4]:
df_16.columns

['Are you self-employed?',
 'How many employees does your company or organization have?',
 'Is your employer primarily a tech company/organization?',
 'Is your primary role within your company related to tech/IT?',
 'Does your employer provide mental health benefits as part of healthcare coverage?',
 'Do you know the options for mental health care available under your employer-provided coverage?',
 'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
 'Does your employer offer resources to learn more about mental health concerns and options for seeking help?',
 'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',
 'If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:',
 'Do you think that discussing a mental health disorder with your employer would have neg

In [5]:
df_17.columns

['#',
 '<strong>Are you self-employed?</strong>',
 'How many employees does your company or organization have?',
 'Is your employer primarily a tech company/organization?',
 'Is your primary role within your company related to tech/IT?',
 'Does your employer provide mental health benefits\xa0as part of healthcare coverage?',
 'Do you know the options for mental health care available under your employer-provided health coverage?',
 'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
 'Does your employer offer resources to learn more about mental health disorders and options for seeking help?',
 'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',
 'If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?',
 'Would you feel more comforta

In [6]:
df_16.show(5, False)

+----------------------+----------------------------------------------------------+-------------------------------------------------------+------------------------------------------------------------+---------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------+---------------------

In [7]:
df_16.count(), len(df_16.columns)

(1433, 63)

In [8]:
df_17.count(),len(df_17.columns)

(756, 123)

In [9]:
df_17.columns

['#',
 '<strong>Are you self-employed?</strong>',
 'How many employees does your company or organization have?',
 'Is your employer primarily a tech company/organization?',
 'Is your primary role within your company related to tech/IT?',
 'Does your employer provide mental health benefits\xa0as part of healthcare coverage?',
 'Do you know the options for mental health care available under your employer-provided health coverage?',
 'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
 'Does your employer offer resources to learn more about mental health disorders and options for seeking help?',
 'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',
 'If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?',
 'Would you feel more comforta

In [10]:
df_16.printSchema()

root
 |-- Are you self-employed?: integer (nullable = true)
 |-- How many employees does your company or organization have?: string (nullable = true)
 |-- Is your employer primarily a tech company/organization?: double (nullable = true)
 |-- Is your primary role within your company related to tech/IT?: double (nullable = true)
 |-- Does your employer provide mental health benefits as part of healthcare coverage?: string (nullable = true)
 |-- Do you know the options for mental health care available under your employer-provided coverage?: string (nullable = true)
 |-- Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?: string (nullable = true)
 |-- Does your employer offer resources to learn more about mental health concerns and options for seeking help?: string (nullable = true)
 |-- Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provi

In [11]:
df_17.printSchema()

root
 |-- #: string (nullable = true)
 |-- <strong>Are you self-employed?</strong>: integer (nullable = true)
 |-- How many employees does your company or organization have?: string (nullable = true)
 |-- Is your employer primarily a tech company/organization?: double (nullable = true)
 |-- Is your primary role within your company related to tech/IT?: double (nullable = true)
 |-- Does your employer provide mental health benefits as part of healthcare coverage?: string (nullable = true)
 |-- Do you know the options for mental health care available under your employer-provided health coverage?: string (nullable = true)
 |-- Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?: string (nullable = true)
 |-- Does your employer offer resources to learn more about mental health disorders and options for seeking help?: string (nullable = true)
 |-- Is your anonymity protected if you choose to take advantage of 

In [12]:
df_16.select("Does your employer offer resources to learn more about mental health concerns and options for seeking help?").show()

+-----------------------------------------------------------------------------------------------------------+
|Does your employer offer resources to learn more about mental health concerns and options for seeking help?|
+-----------------------------------------------------------------------------------------------------------+
|                                                                                                         No|
|                                                                                                        Yes|
|                                                                                                         No|
|                                                                                                       null|
|                                                                                                         No|
|                                                                                                        Yes|
|         

In [13]:
# 2.2.3
df_16.show(5)
df_16.select("Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?").show(10)

+----------------------+----------------------------------------------------------+-------------------------------------------------------+------------------------------------------------------------+---------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------+---------------------

In [14]:
# 2.3 Data exploration
data_16 = df_16.toPandas()
data_17 = df_17.toPandas()

In [15]:
import matplotlib.pyplot as plt
import numpy as np
def stacked_bar_char_with_mh(data, col1, title):
    col2 = "Do you currently have a mental health disorder?"
    df = (data.groupby([col1,col2]).size()/len(data.index)).to_frame("count").reset_index()
    fig, ax = plt.subplots(figsize=(7,8))  
    stacks = data[col2].unique()
    margin_bottom = np.zeros(len(df[col1].unique()))
    colors = ["#ffa600", "#ff6361","#bc5090","#58508d", "#003f5c"]
    for num, answer in enumerate(stacks):
        count = list(df[df[col2] == answer].loc[:, 'count'])

        df[df[col2] == answer].plot.bar(x=col1,y='count', ax=ax, stacked=True, 
                                        bottom = margin_bottom, color=colors[num], label=answer)

        margin_bottom += count

    plt.title(title)
    plt.show()

    
    
# mental health illness family history
col = "Do you have a family history of mental illness?"
title ="mental health disorder family history and mental health disorder diagnose of participants"

stacked_bar_char_with_mh(data_16, col, title)
stacked_bar_char_with_mh(data_17, col, title)

# self-employed
col = "Are you self-employed?"
title_history ="self-employed and mental health disorder diagnose of participants"

stacked_bar_char_with_mh(data_16, col, title)
stacked_bar_char_with_mh(data_17, "<strong>{}</strong>".format(col), title)
#mental health history and mental health disorder
col = "Have you had a mental health disorder in the past?"
title ="mental health disorder history and mental health disorder diagnose of participants"

stacked_bar_char_with_mh(data_16, col, title)
stacked_bar_char_with_mh(data_17, col, title)

<Figure size 700x800 with 1 Axes>

<Figure size 700x800 with 1 Axes>

<Figure size 700x800 with 1 Axes>

<Figure size 700x800 with 1 Axes>

<Figure size 700x800 with 1 Axes>

<Figure size 700x800 with 1 Axes>

In [16]:
#2.4
# 2.4.1find the outliers
df_16.describe("What is your age?").show()
df_17.describe("What is your age?").show()

+-------+------------------+
|summary| What is your age?|
+-------+------------------+
|  count|              1433|
|   mean| 34.28611304954641|
| stddev|11.290930929898892|
|    min|                 3|
|    max|               323|
+-------+------------------+

+-------+-----------------+
|summary|What is your age?|
+-------+-----------------+
|  count|              754|
|   mean|34.98806366047745|
| stddev|8.338051193560366|
|    min|             18.0|
|    max|             67.0|
+-------+-----------------+



In [17]:
numeric_list_16 = [c[0] for c in df_16.dtypes if c[1] !="string"]
numeric_list_16

['Are you self-employed?',
 'Is your employer primarily a tech company/organization?',
 'Is your primary role within your company related to tech/IT?',
 'Do you have medical coverage (private insurance or state-provided) which includes treatment of ??mental health issues?',
 'Do you have previous employers?',
 'Have you ever sought treatment for a mental health issue from a mental health professional?',
 'What is your age?']

In [18]:
numeric_list_17 = [c[0] for c in df_17.dtypes if c[1] !="string"]
numeric_list_17

['<strong>Are you self-employed?</strong>',
 'Is your employer primarily a tech company/organization?',
 'Is your primary role within your company related to tech/IT?',
 'Have you ever discussed your mental health with your employer?',
 'Have you ever discussed your mental health with coworkers?',
 "Have you ever had a coworker discuss their or another coworker's mental health with you?",
 'Overall, how much importance does your employer place on physical health?',
 'Overall, how much importance does your employer place on mental health?',
 'Do you have medical coverage (private insurance or state-provided) that includes treatment of mental health disorders?',
 '<strong>Do you have previous employers?</strong>',
 'Was your employer primarily a tech company/organization?',
 'Did you ever discuss your mental health with your previous employer?',
 'Did you ever discuss your mental health with a previous coworker(s)?',
 "Did you ever have a previous coworker discuss their or another cowork

In [19]:
df_16.describe("What is your age?").show()
df_17.describe("What is your age?").show()

+-------+------------------+
|summary| What is your age?|
+-------+------------------+
|  count|              1433|
|   mean| 34.28611304954641|
| stddev|11.290930929898892|
|    min|                 3|
|    max|               323|
+-------+------------------+

+-------+-----------------+
|summary|What is your age?|
+-------+-----------------+
|  count|              754|
|   mean|34.98806366047745|
| stddev|8.338051193560366|
|    min|             18.0|
|    max|             67.0|
+-------+-----------------+



In [20]:
feature = "What is your age?"
quantiles = [0.25, 0.75]

def find_outliers(df, name):
    quants = df.approxQuantile(feature, quantiles, 0.05)  
    IQR = quants[1] - quants[0] 
    lower = quants[0] - 1.5 * IQR
    upper = quants[1] + 1.5 * IQR
    outliers = df.where((df[feature] < lower) | (df[feature] > upper))
    print("{} outliers detected in {}".format(outliers.count(), name))
    return  lower, upper, outliers.select(feature)
    
lower_16, upper_16, outlier_16 = find_outliers(df_16, "dataset 2016")
outlier_16.show()

45 outliers detected in dataset 2016
+-----------------+
|What is your age?|
+-----------------+
|               55|
|               55|
|               54|
|               56|
|               57|
|               63|
|               55|
|               99|
|               57|
|               55|
|               61|
|               55|
|               54|
|               61|
|               55|
|              323|
|               62|
|               54|
|               55|
|               58|
+-----------------+
only showing top 20 rows



In [21]:
lower_17, upper_17, outlier_17 = find_outliers(df_17, "dataset 2017")
outlier_17.show()

18 outliers detected in dataset 2017
+-----------------+
|What is your age?|
+-----------------+
|             66.0|
|             64.0|
|             61.0|
|             57.0|
|             64.0|
|             62.0|
|             60.0|
|             58.0|
|             59.0|
|             67.0|
|             57.0|
|             60.0|
|             58.0|
|             57.0|
|             56.0|
|             56.0|
|             55.0|
|             57.0|
+-----------------+



In [22]:
#2.4.3 
distinct_gender_16 = df_16.select("What is your gender?").distinct()
print("Unique answers to gender:", distinct_gender_16.count())
distinct_gender_16.show(30)

Unique answers to gender: 71
+--------------------+
|What is your gender?|
+--------------------+
|   genderqueer woman|
|                AFAB|
|Female assigned a...|
|                  fm|
|          non-binary|
|                   F|
|         Genderqueer|
|                 mtf|
|female-bodied; no...|
|                 man|
|                 Man|
|            Cis male|
|         genderqueer|
|                   m|
|             Female |
|I'm a man why did...|
|Female (props for...|
|                null|
|             Agender|
|          Male (cis)|
|                Dude|
|                   f|
|               human|
| Other/Transfeminine|
|                  M||
|               woman|
|        female/woman|
|              Female|
|         Cis female |
|              female|
+--------------------+
only showing top 30 rows



In [23]:
# 3.1 selection
def search_col_by_keywords(df, keywords):
    filter_col = []
    for keyword in keywords:        
        filter_col += [col for col in df.schema.names if keyword in col]
    return filter_col

keywords = ["Describe the conversation", 
            "describe", "Describe", "do you feel", "Disorder", "Stress Response", 
            "why", "Other", "what condition", "If yes", 
            "if you have", "How has it affected your career",
            "#", "Date", "Network ID", "talk to one of us", "anything else", "US state",
            "negative consequences", "anonymity", "prompted you to request", "Overall"]

col_to_filter_out_16 = search_col_by_keywords(df_16, keywords)
col_to_filter_out_16

['Which of the following best describes your work position?',
 'If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?',
 'If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?',
 'Why or why not?',
 'Why or why not?.1',
 'If yes, what condition(s) have you been diagnosed with?',
 'If maybe, what condition(s) do you believe you have?',
 'If so, what condition(s) were you diagnosed with?',
 'If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?',
 'If yes, what condition(s) have you been diagnosed with?',
 'What US state or territory do you live in?',
 'What US state or territory do you work in?',
 'Do you think that discussing a mental health disorder with your employer would have negative consequences?',
 'Do you think that discussing a physical health issue with your employer would have 

In [24]:
col_to_filter_out_17 =search_col_by_keywords(df_17, keywords)
print(col_to_filter_out_17)

['Describe the conversation you had with your employer about your mental health, including their reactions and what actions were taken to address your mental health issue/questions.', 'Describe the conversation with coworkers you had about your mental health including their reactions.', 'Describe the conversation your coworker had with you about their mental health (please do not use names).', 'Describe the conversation you had with your previous employer about your mental health, including their reactions and actions taken to address your mental health issue/questions.', 'Describe the conversation you had with your previous coworkers about your mental health including their reactions.', 'Describe the conversation your coworker had with you about their mental health (please do not use names)..1', 'Briefly describe what you think the industry as a whole and/or employers could do to improve mental health support for employees.', 'Describe the conversation you had with your employer about

In [25]:
print("Shape of 2016 dataset before selection:", (df_16.count(),len(df_16.columns)))
print("Shape of 2017 dataset before selection:", (df_17.count(),len(df_17.columns)))

Shape of 2016 dataset before selection: (1433, 63)
Shape of 2017 dataset before selection: (756, 123)


In [26]:
df_16 = df_16.select([c for c in df_16.columns if c not in  col_to_filter_out_16])
df_17 = df_17.select([c for c in df_17.columns if c not in  col_to_filter_out_17])
print("Shape of 2016 dataset after selection:", (df_16.count(),len(df_16.columns)))
print("Shape of 2017 dataset after selection:", (df_17.count(),len(df_17.columns)))

Shape of 2016 dataset after selection: (1433, 43)
Shape of 2017 dataset after selection: (756, 52)


In [27]:
# 3.2 cleanning data
#3.2.1 outliers and extreme value
from pyspark.sql.functions import mean
mean_age_16 = int(df_16.select(mean(df_16["What is your age?"])).collect()[0][0])
mean_age_17 = int(df_17.select(mean(df_17["What is your age?"])).collect()[0][0])
print(mean_age_16)
print(mean_age_17)

34
34


In [28]:
from pyspark.sql.functions import when, col
col_age = "What is your age?"
df_16 = df_16.withColumn(col_age, when((col(col_age) > upper_16) | (col(col_age) < lower_16), mean_age_16).otherwise(col(col_age)))
df_17 = df_17.withColumn(col_age, when((col(col_age) > upper_17) | (col(col_age) < lower_17), mean_age_17).otherwise(col(col_age)))

In [29]:
df_16.describe("What is your age?").show()
df_17.describe("What is your age?").show()

+-------+-----------------+
|summary|What is your age?|
+-------+-----------------+
|  count|             1433|
|   mean|33.35938590369854|
| stddev| 6.91326533639851|
|    min|               15|
|    max|               53|
+-------+-----------------+

+-------+-----------------+
|summary|What is your age?|
+-------+-----------------+
|  count|              754|
|   mean|34.37533156498674|
| stddev|7.370014007016803|
|    min|             18.0|
|    max|             54.0|
+-------+-----------------+



In [30]:
# 3.2.2 missing value
from pyspark.sql.functions import count, isnull
#2.4.2 identify missing value

def missing_value(df):
    missing = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
    return missing
    
missing_16 = missing_value(df_16)
missing_16.toPandas().rename(index={0: "count"}).T.sort_values("count", ascending =False)

,count
"If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?",1289
Is your primary role within your company related to tech/IT?,1170
Do you believe your productivity is ever affected by a mental health issue?,1146
Do you know local or online resources to seek help for a mental health disorder?,1146
"If you have revealed a mental health issue to a coworker or employee, do you believe this has impacted you negatively?",1146
"If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?",1146
"If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?",1146
Do you have medical coverage (private insurance or state-provided) which includes treatment of ??mental health issues?,1146
Have your observations of how another individual who discussed a mental health disorder made you less likely to reveal a mental health issue yourself in your current workplace?,776
Do you know the options for mental health care available under your employer-provided coverage?,420


In [31]:
missing_17 = missing_value(df_17)
missing_17.toPandas().rename(index={0: "count"}).T.sort_values("count", ascending =False)

,count
"If you have revealed a mental health disorder to a client or business contact, how has this affected you or the relationship?",705
Has being identified as a person with a mental health issue affected your career?,672
Do you have medical coverage (private insurance or state-provided) that includes treatment of mental health disorders?,643
Do you know local or online resources to seek help for a mental health issue?,643
Do you believe your productivity is ever affected by a mental health issue?,643
"If you have revealed a mental health disorder to a coworker or employee, how has this impacted you or the relationship?",643
"<strong>If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?</strong>",643
"<strong>If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?</strong>",643
Have you ever been diagnosed with a mental health disorder?,432
What is your race?,274


In [32]:
# filter out columns completion rate lower than 50
column_to_construct_16 = search_col_by_keywords(df_16, ["have medical coverage", 
                                                        "primary role within your company",
                                                       "mental health benefits as part",
                                                       "primarily a tech company"])
column_to_construct_17 = search_col_by_keywords(df_17, ["have medical coverage", 
                                                        "ever been diagnosed",
                                                        "Does your employer provide mental health benefits",
                                                        "primary role within your company",
                                                        "Is your employer primarily a tech company"])


dict_16 = missing_16.collect()[0].asDict()
col_missing_over_haff_16 = [d for d in dict_16 if ((dict_16[d] > (df_16.count()/2)) & ~(d in column_to_construct_16))] 
df_16 = df_16.select([col for col in df_16.columns if col not in col_missing_over_haff_16])

df_16.count(), len(df_16.columns)

(1433, 36)

In [33]:
dict_17 = missing_17.collect()[0].asDict()
col_missing_over_haff_17 = [d for d in dict_17 if ((dict_17[d] > (df_17.count()/2)) & ~(d in column_to_construct_17))] 
df_17 = df_17.select([col for col in df_17.columns if col not in col_missing_over_haff_17])

df_17.count(), len(df_17.columns)

(756, 45)

In [34]:
missing_value(df_16).toPandas().rename(index={0: "count"}).T.sort_values("count", ascending =False)

,count
Is your primary role within your company related to tech/IT?,1170
Do you have medical coverage (private insurance or state-provided) which includes treatment of ??mental health issues?,1146
Do you know the options for mental health care available under your employer-provided coverage?,420
Is your employer primarily a tech company/organization?,287
Does your employer provide mental health benefits as part of healthcare coverage?,287
"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",287
Does your employer offer resources to learn more about mental health concerns and options for seeking help?,287
Would you feel comfortable discussing a mental health disorder with your coworkers?,287
Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?,287
Do you feel that your employer takes mental health as seriously as physical health?,287


In [35]:
missing_value(df_17).toPandas().rename(index={0: "count"}).T.sort_values("count", ascending =False)

,count
Do you have medical coverage (private insurance or state-provided) that includes treatment of mental health disorders?,643
Have you ever been diagnosed with a mental health disorder?,432
What is your race?,274
Do you know the options for mental health care available under your employer-provided health coverage?,180
Have your observations of how another individual who discussed a mental health issue made you less likely to reveal a mental health issue yourself in your current workplace?,160
Have you ever had a coworker discuss their or another coworker's mental health with you?,118
Would you feel comfortable discussing a mental health issue with your coworkers?,113
How many employees does your company or organization have?,113
Have you ever discussed your mental health with coworkers?,113
Have you ever discussed your mental health with your employer?,113


In [36]:
def cal_mode(df, c):
    mode = df.groupby(c).count().orderBy("count", ascending = False).where(col(c).isNotNull()).first()[0]
    return mode

def fill_na_with_mode(df, construct_list):
    for col in df.columns:
        if col not in construct_list: 
            mode = cal_mode(df, col)
            df = df.na.fill(mode, subset = [col])
    return df

df_16 = fill_na_with_mode(df_16, column_to_construct_16)
df_17 = fill_na_with_mode(df_17, column_to_construct_17)

In [37]:
missing_value(df_16).toPandas().rename(index={0: "count"}).T.sort_values("count", ascending =False)

,count
Is your primary role within your company related to tech/IT?,1170
Do you have medical coverage (private insurance or state-provided) which includes treatment of ??mental health issues?,1146
Is your employer primarily a tech company/organization?,287
Does your employer provide mental health benefits as part of healthcare coverage?,287
Are you self-employed?,0
Do you currently have a mental health disorder?,0
Do you feel that being identified as a person with a mental health issue would hurt your career?,0
Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?,0
How willing would you be to share with friends and family that you have a mental illness?,0
Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?,0


In [38]:
missing_value(df_17).toPandas().rename(index={0: "count"}).T.sort_values("count", ascending =False)

,count
Do you have medical coverage (private insurance or state-provided) that includes treatment of mental health disorders?,643
Have you ever been diagnosed with a mental health disorder?,432
Is your employer primarily a tech company/organization?,113
Is your primary role within your company related to tech/IT?,113
Does your employer provide mental health benefits as part of healthcare coverage?,113
<strong>Are you self-employed?</strong>,0
Do you currently have a mental health disorder?,0
Have you had a mental health disorder in the past?,0
Have you ever sought treatment for a mental health disorder from a mental health professional?,0
Do you have a family history of mental illness?,0


In [39]:
def gender_unique(df):
    distinct_gender = df.select("What is your gender?").distinct()
    print("Unique answers to gender:", distinct_gender.count())
    distinct_gender.show()
    return distinct_gender
    
distinct_gender_16 = gender_unique(df_16)
distinct_gender_17 = gender_unique(df_17)


Unique answers to gender: 70
+--------------------+
|What is your gender?|
+--------------------+
|   genderqueer woman|
|                AFAB|
|Female assigned a...|
|                  fm|
|          non-binary|
|                   F|
|         Genderqueer|
|                 mtf|
|female-bodied; no...|
|                 man|
|                 Man|
|            Cis male|
|         genderqueer|
|                   m|
|             Female |
|I'm a man why did...|
|Female (props for...|
|             Agender|
|          Male (cis)|
|                Dude|
+--------------------+
only showing top 20 rows

Unique answers to gender: 54
+--------------------+
|What is your gender?|
+--------------------+
|  female (cisgender)|
|              femalw|
|            cis-male|
|   My sex is female.|
|                   F|
|                None|
|God King of the V...|
|                 man|
|                 Man|
|            Cis male|
|             Female |
|                   m|
|          Male (ci

In [40]:
gender = "What is your gender?"
def replace_gender(df, to_replace, replaced):
    df = df.withColumn(gender, when(df[gender].isin(to_replace), replaced).otherwise(df[gender]))
    return df

male_16 = ['Male', 'male', 'Male ', 'M', 'm', 'man', 'Cis male', 'Male.','Male (cis)','Man','Sex is male', 'cis male', 'Malr', 'Dude',
  "I'm a man why didn't you make this a drop down question. You should of asked sex? And I would of answered yes please. Seriously how much text can this take? ",
  'mail', 'M|','male ', 'MALE','Cis Male','cisdude','cis man']

male_17 = ['male', 'Male', 'M', 'Man',  'cis-male', 'Mail',  'cis hetero male', 'm',  'Male (cis)',"male (hey this is the tech industry you're talking about)",
    'Cis male', 'man','Cis-male',  'Male, cis', 'cis male ','Male-ish', 'Male ',  'Cis Male', 'dude']

female_16 = ['Female','female', 'I identify as female.','female ', 'Female assigned at birth ', 'F', 'Woman','fm','f', 
    'Cis female ','Female ', 'woman', 'female/woman','Cisgender Female', 'fem', ' Female',
    'Female (props for making this a freeform field, though)','Cis-woman']

female_17 = ['Female','female', 'F','f', 'Female ','Woman', 'femalw', 'femail','female (cis)','My sex is female.',
    'female (cisgender)','Female (cis) ', 'Woman-identified','woman','cis-Female','cis female','F, cisgender', 'Female-ish']


other_16 = ['Bigender', 'non-binary', 'Transitioned, M2F', 'Genderfluid (born female)', 'Other/Transfeminine',
    'Female or Multi-Gender Femme', 'Androgynous', 'male 9:1 female, roughly','nb masculine', 'none of your business', 
    'genderqueer', 'Human', 'Genderfluid','Enby', 'genderqueer woman', 'mtf', 'Queer', 'Agender', 'Fluid','Male/genderqueer', 
    'Nonbinary', 'human', 'Unicorn', 'Male (trans, FtM)', 'Genderqueer','Genderflux demi-girl', 
    'female-bodied; no feelings about gender', 'AFAB', 'Transgender woman']

other_17 = [ 'Genderfluid','Nonbinary','male/androgynous ','uhhhhhhhhh fem genderqueer?','God King of the Valajar',
    'Non-binary','Agender/genderfluid','sometimes','Contextual','Non binary', 'Genderqueer demigirl', 'Genderqueer/non-binary',
    'nonbinary','\\-', 'trans woman','Transfeminine', 'None']

df_16 = replace_gender(df_16, male_16, "Male")
df_16 = replace_gender(df_16, female_16, "Female")
df_16 = replace_gender(df_16, other_16, "Other")

df_17 = replace_gender(df_17, male_17, "Male")
df_17 = replace_gender(df_17, female_17, "Female")
df_17 = replace_gender(df_17, other_17, "Other")


In [41]:
df_16.select(gender).distinct().show()
df_17.select(gender).distinct().show()

+--------------------+
|What is your gender?|
+--------------------+
|              Female|
|               Other|
|                Male|
+--------------------+

+--------------------+
|What is your gender?|
+--------------------+
|              Female|
|               Other|
|                Male|
+--------------------+



In [42]:
# #3.3 constructin new data
employee = column_to_construct_16[2]
self_employed = column_to_construct_16[0]
df_16 = df_16.withColumn(employee, when(col(employee).isNull(), df_16[self_employed]).otherwise(df_16[employee]))\
        .withColumnRenamed(employee, "mental_health_coverage")

In [43]:
it_role = column_to_construct_16[1]
it_company = column_to_construct_16[3]
df_16 = df_16.withColumn(it_company, when(((df_16[it_role] == 0) & (df_16[it_company] == 0)), 0).otherwise(1))\
        .withColumnRenamed(it_company, "IT_industry")

In [44]:
df_16 = df_16.drop(it_role, self_employed)
df_16.select("mental_health_coverage","IT_industry").show(3)
df_16.count(), len(df_16.columns)

+----------------------+-----------+
|mental_health_coverage|IT_industry|
+----------------------+-----------+
|  Not eligible for ...|          1|
|                    No|          1|
|                    No|          1|
+----------------------+-----------+
only showing top 3 rows



(1433, 34)

In [45]:
employee = column_to_construct_17[2]
self_employed = column_to_construct_17[0]
df_17 = df_17.withColumn(employee, when(col(employee).isNull(), df_17[self_employed]).otherwise(df_17[employee]))\
        .withColumnRenamed(employee, "mental_health_coverage")

In [46]:
it_role = column_to_construct_17[3]
it_company = column_to_construct_17[4]
df_17 = df_17.withColumn(it_company, when(((df_17[it_role] == 0) & (df_17[it_company] == 0)), 0).otherwise(1.0))\
        .withColumnRenamed(it_company, "IT_industry")

df_17 = df_17.drop(it_role, self_employed)
df_17.select("mental_health_coverage","IT_industry").show(3)
df_17.count(), len(df_17.columns)

+----------------------+-----------+
|mental_health_coverage|IT_industry|
+----------------------+-----------+
|                    No|        1.0|
|                   Yes|        1.0|
|          I don't know|        1.0|
+----------------------+-----------+
only showing top 3 rows



(756, 43)

In [47]:
df_17 = df_17.withColumn("work_remote", when(df_17["What country do you <strong>live</strong> in?"] == df_17["What country do you <strong>work</strong> in?"], 0).otherwise(1))

In [48]:
df_16.printSchema()

root
 |-- Are you self-employed?: integer (nullable = true)
 |-- How many employees does your company or organization have?: string (nullable = false)
 |-- IT_industry: integer (nullable = false)
 |-- mental_health_coverage: string (nullable = true)
 |-- Do you know the options for mental health care available under your employer-provided coverage?: string (nullable = false)
 |-- Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?: string (nullable = false)
 |-- Does your employer offer resources to learn more about mental health concerns and options for seeking help?: string (nullable = false)
 |-- Would you feel comfortable discussing a mental health disorder with your coworkers?: string (nullable = false)
 |-- Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?: string (nullable = false)
 |-- Do you feel that your employer takes mental health as seriously as 

In [49]:
df_17.printSchema()

root
 |-- <strong>Are you self-employed?</strong>: integer (nullable = true)
 |-- How many employees does your company or organization have?: string (nullable = false)
 |-- IT_industry: double (nullable = false)
 |-- mental_health_coverage: string (nullable = true)
 |-- Do you know the options for mental health care available under your employer-provided health coverage?: string (nullable = false)
 |-- Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?: string (nullable = false)
 |-- Does your employer offer resources to learn more about mental health disorders and options for seeking help?: string (nullable = false)
 |-- Would you feel more comfortable talking to your coworkers about your physical health or your mental health?: string (nullable = false)
 |-- Would you feel comfortable discussing a mental health issue with your direct supervisor(s)?: string (nullable = false)
 |-- Have you ever discusse

In [50]:
from functools import reduce

new_col_16 = ['self-employed', 'company_size',
        'IT_industry','mental_health_coverage',
        'awareness_opt_for_mh_under_coverage',
        'employer_formally_discussed_mh',
        'employer_provide_mh_resources_and_support',
        'comfortable_discuss_mh_with_coworker',
        'comfortable_discuss_mh_with_supervisor',
        'employer_takes_mh_seriously_as_ph',
        'has_prev_employers',
        'prev_employers_provided_mh_benefits',
        'awareness_opt_for_mh_under_coverage_provided_by_prev_employers',
        'prev_employers_formally_discuss_mh',
        'prev_employers_provide_mh_resources_and_support',
        'willingness_to discuss_mh_with_prev_co-workers',
        'willingness_to_discuss_mh_with_supervisor',
        'prev_employer_take_mh_seriously_as_ph',
        'willingness_raise_ph_issue_in_interview',
        'willingness_raise_mh_issue_in_interview',
        'feeling_mh_hurt_career',
        'co-workers_negative_feeling_on_mh_issue',
        'williness_sharing_with_friends_fam_mh_illness',
        'experienced_unsupportive_tomh_issue_in_workplace',
        'family_history','mh_disorder_previous','mh_disorder_current',
        'diagnosed_mh_dsiorder','sought_treatment_from_mh_pro',
        'age', 'gender','country_live', 'country_work',
        'work_remote']

old_col_16 = df_16.schema.names

df_16 = reduce(lambda df_16, idx: df_16.withColumnRenamed(old_col_16[idx], new_col_16[idx]), range(len(old_col_16)), df_16)

In [51]:
new_col_17 = ['self-employed','company_size',
        'IT_industry','mental_health_coverage',
        'awareness_opt_for_mh_under_coverage',
        'employer_formally_discussed_mh',
        'employer_provide_mh_resources_and_support',
        'comfortable_dicusses_coworkers_ph_or_mh',
        'comfortable_discuss_mh_with_supervisor',
        'discussed_mh_with_employer',
        'comfortable_discuss_mh_with_coworker',
        'discussed_mh_with_coworker',
        'discussed_others_mh_with_coworker',
        'has_prev_employers','was_in_tech_industry',
        'prev_employers_provided_mh_benefits',
        'awareness_opt_for_mh_under_coverage_provided_by_prev_employers',
        'prev_employers_formally_discuss_mh',
        'prev_employers_provide_mh_resources_and_support',
        'comfortable_discuss_ph_or_mh_with_prev_employer',
        'willingness_to_discuss_mh_with_supervisor',
        'discussed_mh_with_prev_employer',
        'willingness_to discuss_mh_with_prev_co-workers',
        'discussed_mh_with_prev_coworker',
        'discussed_others_mh_with__prev_coworker',
        'mh_disorder_current','diagnosed_mh_dsiorder',
        'mh_disorder_previous',
        'sought_treatment_from_mh_pro','family_history',
        'other_experience_stop_you_reveal_mh_issue',
        'williness_sharing_with_friends_fam_mh_illness',
        'willingness_raise_ph_issue_in_interview',
        'willingness_raise_mh_issue_in_interview',
        'openly_identified_mh_issue','coworker_react_on_mh',
        'experienced_unsupportive_tomh_issue_in_workplace',
        'experienced_supportive_tomh_issue_in_workplace',
        'age', 'gender','country_live', 'race',
        'country_work', 'work_remote']

old_col_17 = df_17.schema.names
df_17 = reduce(lambda df_17, idx: df_17.withColumnRenamed(old_col_17[idx], new_col_17[idx]), range(len(old_col_17)), df_17)


In [52]:
df_16.printSchema()

root
 |-- self-employed: integer (nullable = true)
 |-- company_size: string (nullable = false)
 |-- IT_industry: integer (nullable = false)
 |-- mental_health_coverage: string (nullable = true)
 |-- awareness_opt_for_mh_under_coverage: string (nullable = false)
 |-- employer_formally_discussed_mh: string (nullable = false)
 |-- employer_provide_mh_resources_and_support: string (nullable = false)
 |-- comfortable_discuss_mh_with_coworker: string (nullable = false)
 |-- comfortable_discuss_mh_with_supervisor: string (nullable = false)
 |-- employer_takes_mh_seriously_as_ph: string (nullable = false)
 |-- has_prev_employers: integer (nullable = true)
 |-- prev_employers_provided_mh_benefits: string (nullable = false)
 |-- awareness_opt_for_mh_under_coverage_provided_by_prev_employers: string (nullable = false)
 |-- prev_employers_formally_discuss_mh: string (nullable = false)
 |-- prev_employers_provide_mh_resources_and_support: string (nullable = false)
 |-- willingness_to discuss_mh_wi

In [53]:
df_17.printSchema()

root
 |-- self-employed: integer (nullable = true)
 |-- company_size: string (nullable = false)
 |-- IT_industry: double (nullable = false)
 |-- mental_health_coverage: string (nullable = true)
 |-- awareness_opt_for_mh_under_coverage: string (nullable = false)
 |-- employer_formally_discussed_mh: string (nullable = false)
 |-- employer_provide_mh_resources_and_support: string (nullable = false)
 |-- comfortable_dicusses_coworkers_ph_or_mh: string (nullable = false)
 |-- comfortable_discuss_mh_with_supervisor: string (nullable = false)
 |-- discussed_mh_with_employer: double (nullable = false)
 |-- comfortable_discuss_mh_with_coworker: string (nullable = false)
 |-- discussed_mh_with_coworker: double (nullable = false)
 |-- discussed_others_mh_with_coworker: double (nullable = false)
 |-- has_prev_employers: integer (nullable = true)
 |-- was_in_tech_industry: double (nullable = false)
 |-- prev_employers_provided_mh_benefits: string (nullable = false)
 |-- awareness_opt_for_mh_under_c

In [54]:
df_16_unique_col = [c for c in df_16.columns if c not in df_17.columns]
df_16_unique_col

['employer_takes_mh_seriously_as_ph',
 'prev_employer_take_mh_seriously_as_ph',
 'feeling_mh_hurt_career',
 'co-workers_negative_feeling_on_mh_issue']

In [55]:
df_17_unique_col = [c for c in df_17.columns if c not in df_16.columns]
df_17_unique_col

['comfortable_dicusses_coworkers_ph_or_mh',
 'discussed_mh_with_employer',
 'discussed_mh_with_coworker',
 'discussed_others_mh_with_coworker',
 'was_in_tech_industry',
 'comfortable_discuss_ph_or_mh_with_prev_employer',
 'discussed_mh_with_prev_employer',
 'discussed_mh_with_prev_coworker',
 'discussed_others_mh_with__prev_coworker',
 'other_experience_stop_you_reveal_mh_issue',
 'openly_identified_mh_issue',
 'coworker_react_on_mh',
 'experienced_supportive_tomh_issue_in_workplace',
 'race']

In [56]:
df_16 = df_16.select([c for c in df_16.columns if c in df_17.columns])
df_17 = df_17.select([c for c in df_17.columns if c in df_16.columns])
df_17 = df_17.select(df_16.columns)

df = df_16.union(df_17)
df.printSchema()

root
 |-- self-employed: integer (nullable = true)
 |-- company_size: string (nullable = false)
 |-- IT_industry: double (nullable = false)
 |-- mental_health_coverage: string (nullable = true)
 |-- awareness_opt_for_mh_under_coverage: string (nullable = false)
 |-- employer_formally_discussed_mh: string (nullable = false)
 |-- employer_provide_mh_resources_and_support: string (nullable = false)
 |-- comfortable_discuss_mh_with_coworker: string (nullable = false)
 |-- comfortable_discuss_mh_with_supervisor: string (nullable = false)
 |-- has_prev_employers: integer (nullable = true)
 |-- prev_employers_provided_mh_benefits: string (nullable = false)
 |-- awareness_opt_for_mh_under_coverage_provided_by_prev_employers: string (nullable = false)
 |-- prev_employers_formally_discuss_mh: string (nullable = false)
 |-- prev_employers_provide_mh_resources_and_support: string (nullable = false)
 |-- willingness_to discuss_mh_with_prev_co-workers: string (nullable = false)
 |-- willingness_to_d

In [57]:
missing_value(df).toPandas().rename(index={0: "count"}).T.sort_values("count", ascending =False)

,count
diagnosed_mh_dsiorder,432
self-employed,0
company_size,0
country_work,0
country_live,0
gender,0
age,0
sought_treatment_from_mh_pro,0
mh_disorder_current,0
mh_disorder_previous,0


In [58]:
col = "diagnosed_mh_dsiorder"
mode = df.groupby(col).count().orderBy("count", ascending = False).first()[0]
df = df.na.fill(mode, col)
df.select([count(when(df[c].isNull(), c)).alias(c) for c in df.schema.names]).\
    toPandas().rename(index={0: "count"}).T.sort_values("count", ascending =False)

,count
self-employed,0
company_size,0
country_work,0
country_live,0
gender,0
age,0
sought_treatment_from_mh_pro,0
diagnosed_mh_dsiorder,0
mh_disorder_current,0
mh_disorder_previous,0


In [59]:
df.printSchema()

root
 |-- self-employed: integer (nullable = true)
 |-- company_size: string (nullable = false)
 |-- IT_industry: double (nullable = false)
 |-- mental_health_coverage: string (nullable = true)
 |-- awareness_opt_for_mh_under_coverage: string (nullable = false)
 |-- employer_formally_discussed_mh: string (nullable = false)
 |-- employer_provide_mh_resources_and_support: string (nullable = false)
 |-- comfortable_discuss_mh_with_coworker: string (nullable = false)
 |-- comfortable_discuss_mh_with_supervisor: string (nullable = false)
 |-- has_prev_employers: integer (nullable = true)
 |-- prev_employers_provided_mh_benefits: string (nullable = false)
 |-- awareness_opt_for_mh_under_coverage_provided_by_prev_employers: string (nullable = false)
 |-- prev_employers_formally_discuss_mh: string (nullable = false)
 |-- prev_employers_provide_mh_resources_and_support: string (nullable = false)
 |-- willingness_to discuss_mh_with_prev_co-workers: string (nullable = false)
 |-- willingness_to_d

In [60]:
# 3.5 formatting
for col in df.columns:
    df.select(col).distinct().show(truncate = False)

+-------------+
|self-employed|
+-------------+
|1            |
|0            |
+-------------+

+--------------+
|company_size  |
+--------------+
|100-500       |
|Jun-25        |
|500-1000      |
|1-May         |
|More than 1000|
|26-100        |
|1-5           |
|6-25          |
+--------------+

+-----------+
|IT_industry|
+-----------+
|0.0        |
|1.0        |
+-----------+

+-------------------------------+
|mental_health_coverage         |
+-------------------------------+
|1.0                            |
|I don't know                   |
|0.0                            |
|Not eligible for coverage / NA |
|Not eligible for coverage / N/A|
|No                             |
|Yes                            |
+-------------------------------+

+-----------------------------------+
|awareness_opt_for_mh_under_coverage|
+-----------------------------------+
|No                                 |
|Yes                                |
|I am not sure                      |
+---------

+------------+
|country_work|
+------------+
|Russia      |
|Sweden      |
|Eritrea     |
|Singapore   |
|Turkey      |
|Germany     |
|Afghanistan |
|Jordan      |
|France      |
|Greece      |
|Slovakia    |
|Argentina   |
|Belgium     |
|Ecuador     |
|Finland     |
|India       |
|China       |
|Chile       |
|Croatia     |
|Italy       |
+------------+
only showing top 20 rows

+-----------+
|work_remote|
+-----------+
|Sometimes  |
|0          |
|1          |
|Never      |
|Always     |
+-----------+



In [61]:
from pyspark.sql.types import IntegerType

df = df.withColumn("IT_industry",df["IT_industry"].cast(IntegerType()))
df = df.withColumn("age",df["age"].cast(IntegerType()))
df.printSchema()

root
 |-- self-employed: integer (nullable = true)
 |-- company_size: string (nullable = false)
 |-- IT_industry: integer (nullable = true)
 |-- mental_health_coverage: string (nullable = true)
 |-- awareness_opt_for_mh_under_coverage: string (nullable = false)
 |-- employer_formally_discussed_mh: string (nullable = false)
 |-- employer_provide_mh_resources_and_support: string (nullable = false)
 |-- comfortable_discuss_mh_with_coworker: string (nullable = false)
 |-- comfortable_discuss_mh_with_supervisor: string (nullable = false)
 |-- has_prev_employers: integer (nullable = true)
 |-- prev_employers_provided_mh_benefits: string (nullable = false)
 |-- awareness_opt_for_mh_under_coverage_provided_by_prev_employers: string (nullable = false)
 |-- prev_employers_formally_discuss_mh: string (nullable = false)
 |-- prev_employers_provide_mh_resources_and_support: string (nullable = false)
 |-- willingness_to discuss_mh_with_prev_co-workers: string (nullable = false)
 |-- willingness_to_d

In [62]:
df = df.withColumn("company_size", when(df["company_size"] == "Jun-25", "6-25").otherwise(when(df["company_size"] == "1-May", "1-5").otherwise(df["company_size"])))
df = df.withColumn("mental_health_coverage", when(df["mental_health_coverage"] == "1.0", "Yes").otherwise(when(df["mental_health_coverage"] == "0.0", "No")\
                    .otherwise(when(df["mental_health_coverage"] == "Not eligible for coverage / NA", "Not eligible for coverage / N/A").otherwise(df["mental_health_coverage"]))))
df = df.withColumn("awareness_opt_for_mh_under_coverage_provided_by_prev_employers", when(df["awareness_opt_for_mh_under_coverage_provided_by_prev_employers"]\
                    .isin(["N/A (was not aware)", "N/A (none offered)","No, I only became aware later","N/A (not currently aware)"]), "No, was not aware")\
                   .otherwise(df["awareness_opt_for_mh_under_coverage_provided_by_prev_employers"]))
df = df.withColumn("willingness_to discuss_mh_with_prev_co-workers", when(df["willingness_to discuss_mh_with_prev_co-workers"] == "At some of my previous employers", "Some of my previous employers")\
                   .otherwise(when(df["willingness_to discuss_mh_with_prev_co-workers"] == "1-May", "1-5").otherwise(df["willingness_to discuss_mh_with_prev_co-workers"])))
df = df.withColumn("willingness_to_discuss_mh_with_supervisor", when(df["willingness_to_discuss_mh_with_supervisor"] == "Yes, at all of my previous employers", "Yes, all of my previous supervisors")\
                    .otherwise(when(df["willingness_to_discuss_mh_with_supervisor"] == "No, at none of my previous employers", "No, none of my previous supervisors").otherwise(when(df["willingness_to_discuss_mh_with_supervisor"] =="Some of my previous employers","Some of my previous supervisors")\
                               .otherwise(df["willingness_to_discuss_mh_with_supervisor"]))))
df = df.withColumn("williness_sharing_with_friends_fam_mh_illness", when(df["williness_sharing_with_friends_fam_mh_illness"].isin(["0","Not open at all"]), "Not open at all")\
                   .otherwise(when(df["williness_sharing_with_friends_fam_mh_illness"].isin(["1", "2", "3","Somewhat not open"]), "Somewhat not open").otherwise(when(df["williness_sharing_with_friends_fam_mh_illness"].isin(["4","5","6","Neutral"]), "Neutral")\
                                         .otherwise(when(df["williness_sharing_with_friends_fam_mh_illness"].isin(["7","8","9","Somewhat open"]), "Somewhat open").otherwise("Very open")))))


df = df.withColumn("mh_disorder_previous", when(df["mh_disorder_previous"].isin("Possibly", "Don't Know"), "Maybe").otherwise(df["mh_disorder_previous"]))
df = df.withColumn("mh_disorder_current", when(df["mh_disorder_current"].isin("Possibly", "Don't Know"), "Maybe").otherwise(df["mh_disorder_current"]))
df = df.withColumn("work_remote", when(df["work_remote"].isin(["Sometimes", "Always"]), "1").otherwise(when(df["work_remote"]=="Never", "0").otherwise(df["work_remote"])))

In [63]:
cols = ["company_size","mental_health_coverage","awareness_opt_for_mh_under_coverage_provided_by_prev_employers","willingness_to discuss_mh_with_prev_co-workers",
        "willingness_to_discuss_mh_with_supervisor","williness_sharing_with_friends_fam_mh_illness", "mh_disorder_previous", "mh_disorder_current"]
for col in cols:
    df.select(col).distinct().show(truncate = False)

+--------------+
|company_size  |
+--------------+
|100-500       |
|500-1000      |
|More than 1000|
|26-100        |
|1-5           |
|6-25          |
+--------------+

+-------------------------------+
|mental_health_coverage         |
+-------------------------------+
|I don't know                   |
|Not eligible for coverage / N/A|
|No                             |
|Yes                            |
+-------------------------------+

+--------------------------------------------------------------+
|awareness_opt_for_mh_under_coverage_provided_by_prev_employers|
+--------------------------------------------------------------+
|Yes, I was aware of all of them                               |
|I was aware of some                                           |
|No, was not aware                                             |
+--------------------------------------------------------------+

+----------------------------------------------+
|willingness_to discuss_mh_with_prev_co-workers|
+-

In [64]:
df.coalesce(1).write.format('com.databricks.spark.csv').save('cleaned.csv',header = 'true')